In [1]:
import qiskit
import numpy
import scipy
# from qiskit.circuit import (
#     ClassicalRegister,
#     Gate,
#     Instruction,
#     Parameter,
#     ParameterExpression,
#     QuantumCircuit,
#     QuantumRegister,
# )

# from qiskit.circuit.library.standard_gates import (
#     CXGate,
#     CYGate,
#     CZGate,
#     RXGate,
#     RXXGate,
#     RYGate,
#     RYYGate,
#     RZGate,
#     RZXGate,
#     RZZGate,
#     XGate,
# )

numpy.set_printoptions(suppress=True)

In [106]:
def nearest_num_qubit(x):
    return int(numpy.ceil(numpy.log2(x)))

def prep_oracle(coeff_array):
    ## See (7.58) in https://arxiv.org/pdf/2201.08309
    ## LCU Oracle for PREPARE for T = \sum_i=0^{K-1} a_i U_i
    ## V|0000...0> = 1/\sqrt(||a||_1) \sum_i=0^{K-1} \sqrt(|a_i|)|i>
    ## where ||a||_1 = \sum_i |a_i|, a_i>0 (WLOG by absorbing the phase into U_i)
    ##                        \sqrt(a_0)    *   *  ...  *
    ## V = 1/\sqrt(||a||_1)   \sqrt(a_1)    *   *  ...  *
    ##                          ....        *   *  ...  *
    ##                        \sqrt(a_{K-1} *   *  ...  *
    
    l1norm = numpy.linalg.norm(coeff_array, ord=1)
    coeff_array_normedsqrt = numpy.sqrt(numpy.abs(coeff_array)/l1norm)
    num_terms = len(coeff_array)
    num_qubits = nearest_num_qubit(num_terms)
    v = numpy.identity(2**num_qubits, dtype=float)
    for row in range(num_terms):
        v[row,0] =  coeff_array_normedsqrt[row]
    ## orthgonoalize v to make it unitary
    q,r = numpy.linalg.qr(v)
    if q[0,0] < 0:
        q = -q
    if numpy.linalg.norm(q[:,0] - v[:,0]) > 1e-12:
        raise ValueError("QR decomposition failed to make V a unitary matrix", "we get", q[:,0], "but suppose to get", v[:,0])
    return q

def select_oracle(unitary_array):
    ## See (7.55) in https://arxiv.org/pdf/2201.08309
    num_terms = len(unitary_array)
    num_qubits_control = nearest_num_qubit(num_terms)
    num_qubits_op = int(numpy.log2(unitary_array[0].shape[0]))
    bin_string = '{0:0'+str(num_qubits_control)+'b}'
    ##
    select_circ = qiskit.QuantumCircuit(num_qubits_control+num_qubits_op)
    for i in range(num_terms):
        ibin = bin_string.format(i)[::-1]
        # ibin = bin_string.format(i)
        control_u = qiskit.quantum_info.Operator(unitary_array[i]).to_instruction().control(num_qubits_control)
        ## For 0 control
        for q in range(len(ibin)):
            qbit = ibin[q]
            if qbit == '0':
                select_circ.x(q)
        ## Apply the controlled-U gate
        select_circ.append( control_u, list(range(num_qubits_control+num_qubits_op)) )
        ## UNDO the X gate
        for q in range(len(ibin)):
            qbit = ibin[q]
            if qbit == '0':
                select_circ.x(q)
    ##
    select_circ.name = 'SELECT'
    return select_circ

def lcu_generator(coeff_array, unitary_array):
    prep_mat = prep_oracle(coeff_array)
    select_circ = select_oracle(unitary_array)
    num_terms = len(unitary_array)
    num_qubits_control = nearest_num_qubit(num_terms)
    num_qubits_op = int(numpy.log2(unitary_array[0].shape[0]))
    ##
    lcu_circ = qiskit.QuantumCircuit(num_qubits_control+num_qubits_op)
    ## Apply the preparation oracle
    lcu_circ.unitary(prep_mat, list(range(num_qubits_control)), label='PREP')
    ## Apply the selection oracle
    lcu_circ.append(select_circ, list(range(num_qubits_control+num_qubits_op)))
    ## Apply the preparation oracle dagger
    lcu_circ.unitary(prep_mat.conj().T, list(range(num_qubits_control)), label='PREP_DAG')
    ##
    lcu_circ.reverse_bits() ## stupid qiskit measurement
    return lcu_circ

In [107]:
test_coefs = numpy.array([1,2,3,4])
test_coefs_normed = test_coefs/numpy.linalg.norm(test_coefs, ord=1)
test_unitaries = [numpy.array([[1,0],[0,1]]), numpy.array([[0,1],[1,0]]), numpy.array([[1,0],[0,-1]]), numpy.array([[0,1j],[1j,0]])]

# test_coefs = numpy.array([1,2])
# test_coefs_normed = test_coefs/numpy.linalg.norm(test_coefs, ord=1)
# test_unitaries = [numpy.array([[1,0],[0,1]]), numpy.array([[0,1],[1,0]])]

## -------------------
if len(test_coefs) != len(test_unitaries):
    raise ValueError("The number of coefficients and unitaries should be the same, but we have", len(test_coefs), "coefficients and", len(test_unitaries), "unitaries")
##
correct_answer = numpy.zeros(test_unitaries[0].shape, dtype=complex)
for i in range(len(test_coefs_normed)):
    correct_answer += test_coefs_normed[i]*test_unitaries[i]

## -------------------
PREP = prep_oracle(test_coefs)
SELE = select_oracle(test_unitaries)
LCU = lcu_generator(test_coefs, test_unitaries)
##
correct_answer

array([[ 0.4+0.j ,  0.2+0.4j],
       [ 0.2+0.4j, -0.2+0.j ]])

In [108]:
PREP, PREP.conj().T@PREP

(array([[ 0.31622777, -0.15811388, -0.27386128,  0.89442719],
        [ 0.4472136 ,  0.89442719, -0.        ,  0.        ],
        [ 0.54772256, -0.27386128,  0.79056942,  0.        ],
        [ 0.63245553, -0.31622777, -0.54772256, -0.4472136 ]]),
 array([[ 1.,  0., -0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [-0.,  0.,  1., -0.],
        [ 0.,  0., -0.,  1.]]))

In [109]:
SELE.draw()

┌───┐           ┌───┐                ┌───┐           ┌───┐           
q_0: ┤ X ├─────■─────┤ X ├──────────■─────┤ X ├─────■─────┤ X ├─────■─────
     ├───┤     │     ├───┤┌───┐     │     ├───┤     │     └───┘     │     
q_1: ┤ X ├─────■─────┤ X ├┤ X ├─────■─────┤ X ├─────■───────────────■─────
     └───┘┌────┴────┐└───┘└───┘┌────┴────┐└───┘┌────┴────┐     ┌────┴────┐
q_2: ─────┤ Unitary ├──────────┤ Unitary ├─────┤ Unitary ├─────┤ Unitary ├
          └─────────┘          └─────────┘     └─────────┘     └─────────┘

In [110]:
LCU.draw()

┌───────┐┌─────────┐┌───────────┐
q_0: ┤0      ├┤0        ├┤0          ├
     │  PREP ││         ││  PREP_DAG │
q_1: ┤1      ├┤1 SELECT ├┤1          ├
     └───────┘│         │└───────────┘
q_2: ─────────┤2        ├─────────────
              └─────────┘

In [111]:
def qiskit_to_normal_order(qiskit_matrix):
    num_qubits = int(numpy.log2(qiskit_matrix.shape[0]))
    bin_str = '{0:0'+str(num_qubits)+'b}'
    new_matrix = numpy.zeros(qiskit_matrix.shape, dtype=complex)
    for i in range(qiskit_matrix.shape[0]):
        for j in range(qiskit_matrix.shape[1]):
            normal_i = int(bin_str.format(i)[::-1],2)
            normal_j = int(bin_str.format(j)[::-1],2)
            new_matrix[normal_i,normal_j] = qiskit_matrix[i,j]
    return new_matrix

circ_op = qiskit_to_normal_order(qiskit.quantum_info.Operator(LCU).data)
circ_op

array([[ 0.4       +0.j        ,  0.2       +0.4j       ,
         0.34641016+0.j        , -0.        -0.34641016j,
        -0.2       -0.j        ,  0.4       -0.2j       ,
         0.28284271+0.j        , -0.        -0.28284271j],
       [ 0.2       +0.4j       , -0.2       -0.j        ,
        -0.        -0.34641016j, -0.51961524-0.j        ,
         0.4       -0.2j       ,  0.1       +0.j        ,
        -0.        -0.28284271j,  0.28284271+0.j        ],
       [ 0.34641016+0.j        , -0.        -0.34641016j,
         0.7       +0.j        ,  0.        +0.3j       ,
        -0.17320508-0.j        ,  0.        +0.17320508j,
        -0.24494897-0.j        ,  0.        +0.24494897j],
       [-0.        -0.34641016j, -0.51961524-0.j        ,
         0.        +0.3j       , -0.55      -0.j        ,
         0.        +0.17320508j,  0.25980762+0.j        ,
         0.        +0.24494897j, -0.24494897+0.j        ],
       [-0.2       -0.j        ,  0.4       -0.2j       ,
        -0

In [112]:
qiskit.quantum_info.Operator(LCU).data

array([[ 0.4       +0.j        , -0.2       -0.j        ,
         0.34641016+0.j        ,  0.28284271+0.j        ,
         0.2       +0.4j       ,  0.4       -0.2j       ,
        -0.        -0.34641016j, -0.        -0.28284271j],
       [-0.2       -0.j        ,  0.1       -0.j        ,
        -0.17320508-0.j        , -0.14142136-0.j        ,
         0.4       -0.2j       ,  0.8       +0.1j       ,
         0.        +0.17320508j,  0.        +0.14142136j],
       [ 0.34641016+0.j        , -0.17320508-0.j        ,
         0.7       +0.j        , -0.24494897-0.j        ,
        -0.        -0.34641016j,  0.        +0.17320508j,
         0.        +0.3j       ,  0.        +0.24494897j],
       [ 0.28284271+0.j        , -0.14142136-0.j        ,
        -0.24494897-0.j        ,  0.8       +0.j        ,
        -0.        -0.28284271j,  0.        +0.14142136j,
         0.        +0.24494897j,  0.        +0.2j       ],
       [ 0.2       +0.4j       ,  0.4       -0.2j       ,
        -0

In [113]:
circ_op[:2,:2]

array([[ 0.4+0.j ,  0.2+0.4j],
       [ 0.2+0.4j, -0.2-0.j ]])

In [114]:
correct_answer

array([[ 0.4+0.j ,  0.2+0.4j],
       [ 0.2+0.4j, -0.2+0.j ]])